In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths
import cv2
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tensorflow.keras.models import load_model
from joblib import dump, load

In [3]:
# load face detector model
detector= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [4]:
# load Facenet model
facenet = load_model('facenet_keras.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# load pretrained svm model
model = load('facenet_svm.joblib')

In [6]:
URL = "http://192.168.178.31:4747/video"
cap = cv2.VideoCapture(0)  # Capture webcam

while (True):
    ret, img = cap.read()
    faces = detector.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
        if  w  >  130 :   

            # create a rectangle margin around the face
            cv2.rectangle(img, (x, y), (x + w, y + h), (128, 128, 128), 1)  # Draw rectangle to main image

            # Crop face
            detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # Crop detected face

            try:
                # Add magin if necessary
                margin = 0
                margin_x = int((w * margin) / 100);
                margin_y = int((h * margin) / 100)
                detected_face = img[int(y - margin_y):int(y + h + margin_y), int(x - margin_x):int(x + w + margin_x)]
                
            except:
                print("detected face has no margin")

            try:
                # detect faces -> resize to 160x160 -> normalize -> predict embeddings -> predict labels
                detected_face = cv2.flip(detected_face, 1)
                image = Image.fromarray(detected_face)
                image = image.resize((160,160))
                fs = np.asarray(image)
                fs = fs.astype('float32')
                mean, std = fs.mean(), fs.std()
                fs = (fs - mean) / std
                sample = np.expand_dims(fs, axis = 0)
                embedding = facenet.predict(sample)
                index = model.predict(embedding)
                
                names = ['Hi Jeska', 'Hi Huyen', 'Hello Mina', 'Hi Chung', 'Hello Nga', 'Hello Kha']
                name = names[index[0]]
                 
                # Tick information frames
                if (True):
                    info_box_color = (46, 200, 255)
                    triangle_cnt = np.array(
                        [(x + int(w / 2), y), (x + int(w / 2) - 20, y - 20), (x + int(w / 2) + 20, y - 20)])
                    cv2.drawContours(img, [triangle_cnt], 0, info_box_color, -1)
                    cv2.rectangle(img, (x + int(w / 2) - 100, y - 20), (x + int(w / 2) + 100, y - 90), info_box_color,
                                  cv2.FILLED)

                    cv2.putText(img, name, (x + int(w / 2) - 80, y - 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 111, 255), 2)

            except Exception as e:
                print("exception", str(e))

    cv2.imshow('img', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press q to quit
        break

# terminate open cv 
cap.release()
cv2.destroyAllWindows()